# Vector RAG vs Graph RAG: a comparison of RAG Workflow w/ Langchain

## 0. Setup

### 0.1. Dependencies

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-chroma neo4j

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv('../.env') # Load environment variables from .env file

False

### 0.2. Setup the model

Setup with OpenAI's `gpt-3.5-turbo-0125` model

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

### 0.3. Prompts

Here are two prompts to test the classification of the document, and to test the summarization capabilities of the chain.

In [4]:
prompts = ["What is this document?","What are the two genetic variations studied in this research and how do they affect the LDL receptor protein?", "How can these findings help improve diagnosis and treatment for Familial Hypercholesterolemia?"]

## 1. Indexing: Load

We use `DocumentLoaders` to load a list of `Documents`.  
A `Document` is an object with some `page_content` (str) and `metadata` (dict).

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

In [6]:
loader = DirectoryLoader("../PMC8467959/chunks/", glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()
docs

[Document(page_content='# TITLE\n\n Characterization of Two Variants at Met 1 of the Human LDLR Gene Encoding the Same Amino Acid but Causing Different Functional Phenotypes\n', metadata={'source': '..\\PMC8467959\\chunks\\0.TITLE.txt'}),
 Document(page_content='# ABSTRACT\n\n Familial hypercholesterolemia (FH) is the most common genetic disorder of lipid metabolism, characterized by increased levels of total and LDL plasma cholesterol, which leads to premature atherosclerosis and coronary heart disease. FH phenotype has considerable genetic heterogeneity and phenotypic variability, depending on LDL receptor activity and lifestyle. To improve diagnosis and patient management, here, we characterized two single nucleotide missense substitutions at Methionine 1 of the human LDLR gene (c.1A>T/p.(Met1Leu) and c.1A>C/p.(Met1Leu)). We used a combination of Western blot, flow cytometry, and luciferase assays to determine the effects of both variants on the expression, activity, and synthesis o

## 2. Indexing: Split

Our loaded document is too long to fit in the context window of many models, and models can struggle to find information in very long inputs. 

To handle this we'll **split the document** into chunks for embedding and vector storage. This should help us retrieve only the most relevant bits of the blog post at run time. In this case we'll split our documents into chunks of 1000 characters with 200 characters of overlap between chunks.

The **overlap** helps mitigate the possibility of separating a statement from important context related to it. We use the `RecursiveCharacterTextSplitter`, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We set `add_start_index=True` so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute `start_index`.

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True 
)

all_splits = text_splitter.split_documents(docs)

In [9]:
print(len(all_splits),"splits")
print(len(all_splits[2].page_content),"characters")
print(all_splits[2].page_content)
print(all_splits[2].metadata)

54 splits
992 characters
Familial hypercholesterolemia (FH) is the most common genetic disorder of lipid metabolism, characterized by increased levels of total and LDL plasma cholesterol, which leads to premature atherosclerosis and coronary heart disease. FH phenotype has considerable genetic heterogeneity and phenotypic variability, depending on LDL receptor activity and lifestyle. To improve diagnosis and patient management, here, we characterized two single nucleotide missense substitutions at Methionine 1 of the human LDLR gene (c.1A>T/p.(Met1Leu) and c.1A>C/p.(Met1Leu)). We used a combination of Western blot, flow cytometry, and luciferase assays to determine the effects of both variants on the expression, activity, and synthesis of LDLR. Our data show that both variants can mediate translation initiation, although the expression of variant c.1A>T is very low. Both variants are in the translation initiation codon and codify for the same amino acid p.(Met1Leu), yet they lead to di

## 3. Vector RAG

In [10]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

### 3.1. Indexing: Store

In [11]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

### 3.2. Retrieve

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [13]:
# EXAMPLE
retrieved_docs = retriever.invoke(prompts[0])
retrieved_docs

[Document(page_content='# ABSTRACT', metadata={'source': '..\\PMC8467959\\chunks\\1.ABSTRACT.txt', 'start_index': 0}),
 Document(page_content='# ACK_FUND\n\n Funding\n\n This research received no external funding.\n\n Institutional Review Board Statement\n\n The study was conducted according to the guidelines of the Declaration of Helsinki, and the Portuguese FH Study was approved by the National Institute of Health Ethics Committee and the National Data Protection Commission on 21 September 2009, protocol code 3962/2009.\n\n Informed Consent Statement\n\n Informed consent was obtained from all subjects involved in the study.', metadata={'source': '..\\PMC8467959\\chunks\\8.ACK_FUND.txt', 'start_index': 0}),
 Document(page_content='# INTRO\n\n 1. Introduction', metadata={'source': '..\\PMC8467959\\chunks\\2.INTRO.txt', 'start_index': 0}),
 Document(page_content='Publisher’s Note: MDPI stays neutral with regard to jurisdictional claims in published maps and institutional affiliations.',

### 3.3. Generate

In [14]:
template = """You are an assistant for question-answering tasks on a scientific paper, including classification and summarization of this paper.
Use the following pieces of context to answer the question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}

ANSWER:"""

prompt_vector_rag = PromptTemplate.from_template(template)

We’ll use the LCEL Runnable protocol to define the chain, allowing us to - pipe together components and functions in a transparent way - automatically trace our chain in LangSmith - get streaming, async, and batched calling out of the box

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_vector_rag
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [16]:
result_vec = rag_chain_with_source.invoke(prompts[2])

with open('result_vector_rag.txt', 'w') as file:
    file.write(str(result_vec))

result_vec

{'context': [Document(page_content='Familial hypercholesterolemia: A complex genetic disease with variable phenotypes\n\n Genetic identification of familial hypercholesterolemia within a single U.S. Health care system\n\n Mutational analysis and genotype-phenotype relation in familial hypercholesterolemia: The SAFEHEART registry\n\n Diagnostic Yield of Sequencing Familial Hypercholesterolemia Genes in Severe Hypercholesterolemia\n\n Attainment of LDL-cholesterol treatment goals in patients with familial hypercholesterolemia: 5-year SAFEHEART registry follow-up\n\n Presence and type of low density lipoprotein receptor (LDLR) mutation influences the lipid profile and response to lipid-lowering therapy in Brazilian patients with heterozygous familial hypercholesterolemia\n\n Molecular genetics of the LDL receptor gene in familial hypercholesterolemia\n\n Update of the Portuguese Familial Hypercholesterolaemia Study', metadata={'source': '..\\PMC8467959\\chunks\\11.REF.txt', 'start_index':

## 4. Graph RAG

In [17]:
from langchain_community.graphs import Neo4jGraph
from neo4j.exceptions import ClientError
from langchain_experimental.graph_transformers import LLMGraphTransformer

### 4.0. Setup Graph

In [23]:
graph = Neo4jGraph()

### 4.1. Index

In [24]:
# Function to reset the graph
def reset_graph(graph):
    try:
        # Cypher query to delete all nodes and relationships
        graph.query("MATCH (n) DETACH DELETE n")
        print("The Graph has been reset.")
    except ClientError as e:
        print(f"Client error during graph reset: {e}")
    except Exception as e:
        print(f"An error occurred during graph reset: {e}")

# Reset the graph
reset_graph(graph)

The Graph has been reset.


In [39]:
all_splits

[Document(page_content='# TITLE\n\n Characterization of Two Variants at Met 1 of the Human LDLR Gene Encoding the Same Amino Acid but Causing Different Functional Phenotypes', metadata={'source': '..\\PMC8467959\\chunks\\0.TITLE.txt', 'start_index': 0, 'id': '23ad1f11370f46bf251982cd2b3e24d6'}),
 Document(page_content='# ABSTRACT', metadata={'source': '..\\PMC8467959\\chunks\\1.ABSTRACT.txt', 'start_index': 0, 'id': '156810d7cc8a2fcb362480ad3d2eed39'}),
 Document(page_content='Familial hypercholesterolemia (FH) is the most common genetic disorder of lipid metabolism, characterized by increased levels of total and LDL plasma cholesterol, which leads to premature atherosclerosis and coronary heart disease. FH phenotype has considerable genetic heterogeneity and phenotypic variability, depending on LDL receptor activity and lifestyle. To improve diagnosis and patient management, here, we characterized two single nucleotide missense substitutions at Methionine 1 of the human LDLR gene (c.1

In [25]:
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(all_splits)


Prompts
 [ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='# Knowledge Graph Instructions for GPT-4\n## 1. Overview\nYou are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.\nTry to capture as much information from the text as possible without sacrifing accuracy. Do not add any information that is not explicitly mentioned in the text\n- **Nodes** represent entities and concepts.\n- The aim is to achieve simplicity and clarity in the knowledge graph, making it\naccessible for a vast audience.\n## 2. Labeling Nodes\n- **Consistency**: Ensure you use available types for node labels.\nEnsure you use basic or elementary types for node labels.\n- For example, when you identify an entity representing a person, always label it as **\'person\'**. Avoid using more specific terms like \'mathematician\' or \'scientist\'  - **Node IDs**: Never utilize int

In [26]:
for doc in graph_documents:
    for node in doc.nodes:
        if '_' in node.id:
            node.id = node.id.replace('_',' ')
        if '_' in node.type:
            node.type = node.type.replace('_',' ')

In [27]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

### 4.2. Retrieve

#### 4.2.1 Detect entities in the user input

In [28]:
nodes = graph.query("MATCH (n) RETURN DISTINCT n.id AS id, labels(n) AS types")
nodes

[{'id': 'Human Ldlr Gene', 'types': ['__Entity__', 'Gene']},
 {'id': 'Met 1', 'types': ['__Entity__', 'Amino acid']},
 {'id': 'Machine Learning', 'types': ['__Entity__', 'Concept']},
 {'id': 'Deep Learning', 'types': ['__Entity__', 'Concept']},
 {'id': 'Neural Networks', 'types': ['__Entity__', 'Concept']},
 {'id': 'Familial Hypercholesterolemia',
  'types': ['__Entity__',
   'Genetic disorder',
   'Disease',
   'Condition',
   'Disorder']},
 {'id': 'Ldl Plasma Cholesterol', 'types': ['__Entity__', 'Metabolite']},
 {'id': 'Atherosclerosis', 'types': ['__Entity__', 'Disease', 'Condition']},
 {'id': 'Coronary Heart Disease',
  'types': ['__Entity__', 'Disease', 'Condition']},
 {'id': 'Ldlr Gene', 'types': ['__Entity__', 'Gene']},
 {'id': 'C.1A>T',
  'types': ['__Entity__',
   'Genetic variant',
   'Variant',
   'Entity',
   'Mutation',
   'Genetic variation']},
 {'id': 'C.1A>C',
  'types': ['__Entity__',
   'Genetic variant',
   'Variant',
   'Entity',
   'Mutation',
   'Genetic variatio

In [29]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field()

prompt_vector_rag = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting entities from the text.\n",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt_vector_rag | llm.with_structured_output(Entities)

In [30]:
entities = entity_chain.invoke({"entities": nodes, "question": prompts[2]})
print(prompts[2])
entities

How can these findings help improve diagnosis and treatment for Familial Hypercholesterolemia?


Entities(names=['findings', 'diagnosis', 'treatment', 'Familial Hypercholesterolemia'])

#### 4.2.2. Search with Full Text Index

In [31]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [32]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        
        result += "\n".join([el['output'] for el in response])
    return result

In [33]:
print(structured_retriever(prompts[2]))

Familial Hypercholesterolemia - AFFECTS -> Ldl Plasma Cholesterol
Familial Hypercholesterolemia - LEADS_TO -> Atherosclerosis
Familial Hypercholesterolemia - LEADS_TO -> Coronary Heart Disease
Familial Hypercholesterolemia - ASSOCIATED_WITH -> Ldlr
Familial Hypercholesterolemia - ASSOCIATED_WITH -> Apob
Familial Hypercholesterolemia - ASSOCIATED_WITH -> Pcsk9
Familial Hypercholesterolemia - ASSOCIATED_WITH -> Abcg5/8
Familial Hypercholesterolemia - ASSOCIATED_WITH -> Apoe
Familial Hypercholesterolemia - ASSOCIATED_WITH -> Ldlrap1
Familial Hypercholesterolemia - ASSOCIATED_WITH -> Lipa
Familial Hypercholesterolemia - PROMOTES -> Atherosclerosis
Familial Hypercholesterolemia - PROMOTES -> Coronary Heart Disease
Familial Hypercholesterolemia - GENETIC_IDENTIFICATION -> U.S. Health Care System
Familial Hypercholesterolemia - MOLECULAR_GENETICS -> Ldl Receptor Gene
Familial Hypercholesterolemia - GENOTYPE-PHENOTYPE_RELATION -> Safeheart Registry
Familial Hypercholesterolemia - IDENTIFIED_AS

#### 4.2.3. Custom Cypher generating chain

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Generate Cypher statement based on natural language input
cypher_template = """Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question:
{schema}
Relationships in the question map to the following database values:
{relationships_list}
Question: {question}
Cypher query:"""

cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, convert it to a simple Cypher query. Use available entities only. Else do not specify type or name or id, just use a generic node. No pre-amble. Use backticks to specify the type Do not specify the node type, just the id. Be concise and do not make more than one MATCH Return statement should always be: RETURN *",
        ),
        ("human", cypher_template),
    ]
)

cypher_chain = (
    RunnablePassthrough.assign(names=entity_chain)
    | RunnablePassthrough.assign(
        relationships_list=lambda x: structured_retriever(x["names"]),
        schema=lambda _: graph.get_schema,
    )
    | cypher_prompt
    | llm.bind(stop=["\nCypherResult:"])
    | StrOutputParser()
)

In [35]:
cypher_query = cypher_chain.invoke({"question": prompts[2]})
cypher_query

'MATCH (g:Gene)-[:GENETICS]->(gd:Genetic disorder {id: "Familial Hypercholesterolemia"})-[:GENOTYPE-PHENOTYPE_RELATION]->(r:Registry)-[:TREATMENT_FOLLOW-UP]->(c:Cholesterol)\nMATCH (gd)-[:GENETIC_IDENTIFICATION]->(h:Healthcare system {id: "U.S. Health Care System"})\nMATCH (gd)-[:DIAGNOSTIC_SEQUENCING]->(ch:Cholesterol)\nMATCH (gd)-[:IS_ASSOCIATED_WITH]->(g1:Gene)-[:IS_ASSOCIATED_WITH]->(g2:Protein)-[:IS_ASSOCIATED_WITH]->(g3:Coding sequence)\nMATCH (gd)-[:HAS_OMIM_ID]->(o:Omim {id: "143890"})\nMATCH (gd)-[:CAUSES]->(m:Metabolite)\nMATCH (gd)-[:LEADS_TO]->(c1:Condition {id: "Atherosclerosis"})\nMATCH (gd)-[:LEADS_TO]->(c2:Condition {id: "Coronary Heart Disease"})\nRETURN *'

In [36]:
from langchain.agents import AgentExecutor
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

# Define prompts for Cypher execution and correction
cypher_prompt = PromptTemplate(template="Write a Cypher query to answer the following question: {question}", input_variables=['question'])
correction_prompt = PromptTemplate(template="The Cypher query resulted in an error: {error}. Based on the error, give a corrected Cypher query. For types containing space, surround them with backticks, otherwise it would make an error you won't detect after. Make uses of Relationships at maximum. Remember to return a variable. Do not add comment, nor any other text, just the query.", input_variables=['error'])
chain_cypher_query = GraphCypherQAChain.from_llm(llm, graph=graph)
chain_cypher_correct = llm | StrOutputParser()

def validate_cypher(cypher_query):
  try:
    # Run the chain and capture the output/error
    output = chain_cypher_query.invoke(cypher_query)
    return output
  except Exception as e:
    print("[Error]", e)
    return f"Error: {str(e)}"

def correct_cypher(error_message, original_query):
  # Use LLM or rule-based system to suggest correction based on error message
  corrected_query = chain_cypher_correct.invoke(correction_prompt.format(question=original_query, error=error_message))
  return corrected_query

def iterative_cypher_validation(initial_query):
  current_query = initial_query
  while True:
    output = validate_cypher(current_query)
    if isinstance(output, str) and output.startswith("Error"):
      correction = correct_cypher(output, current_query)
      current_query = correction
    else:
      return output

final_result = iterative_cypher_validation(cypher_query)
print(final_result['query'])
print("=================")
graph.query(final_result['query'])

[Error] Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'disorder': expected ")", "WHERE", "{" or a parameter (line 1, column 41 (offset: 40))
"MATCH (g:Gene)-[:GENETICS]->(gd:Genetic disorder {id: "Familial Hypercholesterolemia"})-[:GENOTYPE-PHENOTYPE_RELATION]->(r:Registry)-[:TREATMENT_FOLLOW-UP]->(c:Cholesterol)"
                                         ^}
[Error] Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '-': expected
  "*"
  "WHERE"
  "]"
  "{"
  a parameter (line 1, column 101 (offset: 100))
"MATCH (g:Gene)-[:GENETICS]->(gd:`Genetic disorder` {id: "Familial Hypercholesterolemia"})-[:GENOTYPE-PHENOTYPE_RELATION]->(r:Registry)-[:TREATMENT_FOLLOW-UP]->(c:Cholesterol)"
                                                                                                     ^}
[Error] Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.Synta

[]

#### 4.2.4. Generating answers based on database results

In [37]:
from langchain.chains.graph_qa.cypher_utils import CypherQueryCorrector, Schema

# Cypher validation tool for relationship directions
corrector_schema = [
    Schema(el["start"], el["type"], el["end"])
    for el in graph.structured_schema.get("relationships")
]
cypher_validation = CypherQueryCorrector(corrector_schema)

def log_cypher_query(query):
    print(f"Generated Cypher Query: \n{query}")
    return query

# Generate natural language response based on database results
response_template = """Based on the the question, Cypher query, and Cypher response, write a natural language response:
Question: {question}
Cypher query: {query}
Cypher Response: {response}"""  # noqa: E501

response_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and Cypher response, convert it to a natural"
            " language answer. No pre-amble.",
        ),
        ("human", response_template),
    ]
)

graph_rag_chain = (
    RunnablePassthrough.assign(query=cypher_chain)
    | RunnablePassthrough.assign(query=lambda x: log_cypher_query(x["query"]))
    | RunnablePassthrough.assign(query=lambda x: iterative_cypher_validation(x["query"])["query"])
    | RunnablePassthrough.assign(response=lambda x: graph.query(x["query"]))
    | response_prompt
    | llm
    | StrOutputParser()
)

In [38]:
graph_rag_chain.invoke({"question": prompts[2]})

Generated Cypher Query: 
MATCH (f:Genetic disorder {id: "Familial Hypercholesterolemia"})-[:GENOTYPE-PHENOTYPE_RELATION]->(r:Registry),
(f)-[:GENOTYPE-PHENOTYPE_RELATION]->(g:Gene),
(f)-[:GENOTYPE-PHENOTYPE_RELATION]->(t:Treatment),
(f)-[:GENOTYPE-PHENOTYPE_RELATION]->(m:Management)
RETURN *
[Error] Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'disorder': expected ")", "WHERE", "{" or a parameter (line 1, column 18 (offset: 17))
"MATCH (f:Genetic disorder {id: "Familial Hypercholesterolemia"})-[:GENOTYPE-PHENOTYPE_RELATION]->(r:Registry),"
                  ^}
[Error] Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '-': expected
  "*"
  "WHERE"
  "]"
  "{"
  a parameter (line 1, column 78 (offset: 77))
"MATCH (f:`Genetic disorder` {id: "Familial Hypercholesterolemia"})-[:GENOTYPE-PHENOTYPE_RELATION]->(r:Registry)"
                                               

'These findings may not be able to help improve diagnosis and treatment for Familial Hypercholesterolemia as there were no results found in the database for this query.'

## Hybrid RAG